In [117]:
import os, glob
import subprocess
import tempfile
from tqdm import tqdm
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
from rioxarray import merge
from osgeo import gdal
from joblib import Parallel, delayed
from oggm import utils
import gzip
import json
import topojson
import shapely
from shapely.geometry import Polygon, MultiPolygon
from shapely.wkt import loads, dumps

In [114]:
# Define a function to round the coordinates of geometries
def extract_polygons(geometry, decimals=5):
    """Extract all polygons from a geometry, treating every ring (exterior + interior) as separate polygons."""
    polygons = []

    if isinstance(geometry, Polygon):
        # Convert exterior ring into a separate polygon
        exterior_coords = [(round(x, decimals), round(y, decimals)) for x, y in geometry.exterior.coords]
        polygons.append(Polygon(exterior_coords))

        # Convert each interior ring (hole) into a separate polygon
        for interior in geometry.interiors:
            interior_coords = [(round(x, decimals), round(y, decimals)) for x, y in interior.coords]
            if len(interior_coords) > 3:  # Ensure valid polygon
                polygons.append(Polygon(interior_coords))

    elif isinstance(geometry, MultiPolygon):
        for poly in geometry.geoms:
            polygons.extend(extract_polygons(poly, decimals))

    return polygons

In [123]:
# Import .shp data and convert to .geojson
version = '62'

for rgi in range(1,20):

    FILE_SHP_RGI = utils.get_rgi_region_file(region=f"{rgi:02d}", version=version)
    print(rgi, FILE_SHP_RGI)

    gdf = gpd.read_file(FILE_SHP_RGI)

    gdf = gdf[['geometry']]

    gdf_rounded = gdf.copy()

    gdf_rounded['geometry'] = gdf_rounded['geometry'].apply(lambda geom: shapely.set_precision(geom, 0.00001))

    
    OUT_PATH = f"/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI{version}/geometries"

    geojson_file = f"{OUT_PATH}/" + FILE_SHP_RGI.rsplit('/', 1)[-1].replace('.shp', '_cleaned.geojson')
    print(OUT_PATH)
    print(geojson_file)
    gdf_rounded.to_file(geojson_file, driver='GeoJSON')
    
    tippecanoe_path_out = "/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries/"
    tippecanoe_file_out = f"geoms_rgi{version}_rgi{rgi}.mbtiles"
    tippecanoe_out_layer_name = f"geoms_rgi{version}_rgi{rgi}"
    tippecanoe_out = tippecanoe_path_out + tippecanoe_file_out

    # Construct the Tippecanoe command
    tippecanoe_command = f"tippecanoe -o {tippecanoe_out} -l {tippecanoe_out_layer_name} -zg {geojson_file} --force"

    # Run Tippecanoe command via subprocess
    subprocess.run(tippecanoe_command, shell=True, check=True)


1 /home/maffe/OGGM/rgi/RGIV62/01_rgi62_Alaska/01_rgi62_Alaska.shp
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries/01_rgi62_Alaska_cleaned.geojson


27108 features, 35224161 bytes of geometry, 27124 bytes of separate metadata, 0 bytes of string pool
Choosing a maxzoom of -z5 for features about 11461 feet (3494 meters) apart
Choosing a maxzoom of -z10 for resolution of about 253 feet (77 meters) within features
  99.9%  10/113/296  


2 /home/maffe/OGGM/rgi/RGIV62/02_rgi62_WesternCanadaUS/02_rgi62_WesternCanadaUS.shp
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries/02_rgi62_WesternCanadaUS_cleaned.geojson


18855 features, 9373068 bytes of geometry, 18871 bytes of separate metadata, 0 bytes of string pool
Choosing a maxzoom of -z5 for features about 9117 feet (2779 meters) apart
Choosing a maxzoom of -z11 for resolution of about 237 feet (72 meters) within features
  99.9%  11/290/571  


3 /home/maffe/OGGM/rgi/RGIV62/03_rgi62_ArcticCanadaNorth/03_rgi62_ArcticCanadaNorth.shp
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries/03_rgi62_ArcticCanadaNorth_cleaned.geojson


...

15908 features, 11533813 bytes of geometry, 15924 bytes of separate metadata, 0 bytes of string pool
Choosing a maxzoom of -z6 for features about 7989 feet (2435 meters) apart
Choosing a maxzoom of -z11 for resolution of about 250 feet (76 meters) within features
  99.9%  11/613/1297  


18 /home/maffe/OGGM/rgi/RGIV62/18_rgi62_NewZealand/18_rgi62_NewZealand.shp
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries/18_rgi62_NewZealand_cleaned.geojson


3537 features, 1245612 bytes of geometry, 3553 bytes of separate metadata, 0 bytes of string pool
Choosing a maxzoom of -z6 for features about 4327 feet (1319 meters) apart
Choosing a maxzoom of -z11 for resolution of about 245 feet (74 meters) within features
  99.9%  11/1997/1296  


19 /home/maffe/OGGM/rgi/RGIV62/19_rgi62_AntarcticSubantarctic/19_rgi62_AntarcticSubantarctic.shp
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries
/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI62/geometries/19_rgi62_AntarcticSubantarctic_cleaned.geojson


2752 features, 6760775 bytes of geometry, 2768 bytes of separate metadata, 0 bytes of string pool
Choosing a maxzoom of -z4 for features about 27058 feet (8248 meters) apart
Choosing a maxzoom of -z10 for resolution of about 355 feet (108 meters) within features
  99.9%  10/409/698  
